# Model eval: WBGT in models v. ISIMIP obs get matching periods in each model

For periods 1990-2019, 2000-2019, 2010-2019 in obs, what is corresponding period in each model (for model evaluation)

In [1]:
# necessary: netcdf4-python, regionmask, (xarray, geopandas), openpyxl?, cartopy?

import numpy as np
import pandas as pd
import os, glob, re 
import math
import xarray as xr
import netCDF4
import time
from datetime import timedelta, datetime
import matplotlib.pyplot as plt

# My settings and functions
from settings import *
start_message() 

from functions import *
print(datetime.now(), 'fxns imported')

flags: {'models': 'ISIMIP3b', 'metric': 'WBGT', 'method': 'calculate', 'time_method': None, 'chunk_version': 2}
var: None
VARs: ['tasmax', 'huss', 'ps']
indir_p: /data/brussel/vo/000/bvo00012/data/dataset/ISIMIP/ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/
indir_s: /data/brussel/vo/000/bvo00012/data/dataset/ISIMIP/ISIMIP3b/SecondaryInputData/climate/atmosphere/bias-adjusted/global/daily/
indir_obs: /data/brussel/vo/000/bvo00012/data/dataset/ISIMIP/ISIMIP3a/InputData/climate/atmosphere/obsclim/global/daily/historical/
GCMs: ['CanESM5', 'CNRM-CM6-1', 'GFDL-ESM4', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0']
datasets: ['GSWP3-W5E5', '20CRv3', '20CRv3-ERA5', '20CRv3-W5E5']
lat_chunk: 90
lon_chunk: 120
time_chunk: -1
outdirs: /data/brussel/vo/000/bvo00012/vsc10419/attr-hw/output
figdirs: /data/brussel/vo/000/bvo00012/vsc10419/attr-hw/figures
start_pi: 1850
end_pi: 1900
target_years: None
observed_warming_path: ../../data/gmst/gmst-obs/forster2023/decadal_averages.csv
dir_gms

In [18]:
outdir=os.path.join('..','..', 'output', 'output_df-gmt-mapping')

In [2]:
%matplotlib inline

In [3]:
df_gmst_mod = merge_model_gmst(GCMs, dir_gmst_models)
df_gmst_mod

,CanESM5,CNRM-CM6-1,GFDL-ESM4,IPSL-CM6A-LR,MIROC6,MRI-ESM2-0
year,,,,,,
1850,0.096275,0.07598,-0.094941,-0.184196,-0.068196,0.062608
1851,-0.149725,0.03998,-0.151941,-0.016196,-0.095196,0.087608
1852,-0.101725,0.00998,0.027059,-0.032196,-0.079196,0.021608
1853,0.038275,0.03198,0.061059,0.036804,-0.019196,-0.052392
1854,-0.075725,0.03098,-0.113941,0.107804,0.191804,-0.107392
...,...,...,...,...,...,...
2096,6.090275,4.75998,3.465059,5.281804,3.370804,3.743608
2097,6.278275,4.75398,3.466059,5.512804,3.036804,3.795608
2098,6.364275,4.95798,3.442059,5.356804,3.112804,3.624608


In [4]:

# annual obs warming
df_gmst_obs_annual = pd.read_csv(os.path.join('../../data/gmst/gmst-obs/forster2023/annual_averages.csv')).rename(
    columns={'timebound_lower':'year'}).set_index('year')[['gmst']]

df_gmst_obs_annual

,gmst
year,
1850,-0.053137
1851,0.056863
1852,0.084363
1853,0.049363
1854,0.049363
...,...
2018,1.109363
2019,1.236863
2020,1.269363


In [9]:
da_10 = calc_warming_periods_models_all_years(GCMs, 
                                          dir_gmst_models, 
                                          observed_warming_path=os.path.join('../../data/gmst/gmst-obs/forster2023/annual_averages.csv'),
                                          method='window',
                                          window=10,
                                          min_periods=1,
                                          flatten=False).sel(year=2019)

da_20 = calc_warming_periods_models_all_years(GCMs, 
                                          dir_gmst_models, 
                                          observed_warming_path=os.path.join('../../data/gmst/gmst-obs/forster2023/annual_averages.csv'),
                                          method='window',
                                          window=20,
                                          min_periods=1,
                                          flatten=False).sel(year=2019)

da_30 = calc_warming_periods_models_all_years(GCMs, 
                                          dir_gmst_models, 
                                          observed_warming_path=os.path.join('../../data/gmst/gmst-obs/forster2023/annual_averages.csv'),
                                          method='window',
                                          window=30,
                                          min_periods=1,
                                          flatten=False).sel(year=2019)

In [10]:
da_10.to_pandas()

feature,year_mod,temp
model,,
obs,2019,1.074113
CanESM5,2004,1.085475
CNRM-CM6-1,2021,1.08708
GFDL-ESM4,2029,1.069159
IPSL-CM6A-LR,2009,1.058504
MIROC6,2034,1.096404
MRI-ESM2-0,2021,1.076008


In [11]:
da_20.to_pandas()

feature,year_mod,temp
model,,
obs,2019,0.968738
CanESM5,2008,0.971625
CNRM-CM6-1,2021,0.96888
GFDL-ESM4,2029,0.962609
IPSL-CM6A-LR,2010,0.964854
MIROC6,2034,0.956754
MRI-ESM2-0,2022,0.962758


In [12]:
da_30.to_pandas()

feature,year_mod,temp
model,,
obs,2019,0.868529
CanESM5,2009,0.874408
CNRM-CM6-1,2020,0.858314
GFDL-ESM4,2029,0.859925
IPSL-CM6A-LR,2011,0.869971
MIROC6,2033,0.862037
MRI-ESM2-0,2024,0.878141


In [17]:
df_join = pd.DataFrame(
    data = {'10_yr': da_10.to_pandas().year_mod, 
            '20_yr': da_20.to_pandas().year_mod, 
            '30_yr': da_30.to_pandas().year_mod},
    index=da_10.to_pandas().index
)
df_join    

,10_yr,20_yr,30_yr
model,,,
obs,2019,2019,2019
CanESM5,2004,2008,2009
CNRM-CM6-1,2021,2021,2020
GFDL-ESM4,2029,2029,2029
IPSL-CM6A-LR,2009,2010,2011
MIROC6,2034,2034,2033
MRI-ESM2-0,2021,2022,2024


In [19]:
#df_join.to_csv(os.path.join(outdir,'df_gmt_mapping_endyear2019_eval.csv'))